# Thalamic nuclei mean kurtosis estimation

> Import modules

In [1]:
import pandas
import os
import re
import numpy
import nibabel
import argparse
import textwrap
import time
import pp

> Set input subjects / file locations

In [2]:
dataLoc = os.getcwd()
subjects = [x for x in os.listdir(dataLoc) if x.startswith('N') or x.startswith('F')]

In [3]:
def get_segment_map(subject,side, dataLoc):
    '''
    Returns nibabel matrix of 
    the connectivity-based segmentation map of thalamus
    '''
    segment_file = os.path.join(dataLoc,
                              subject,
                              'segmentation',
                              side,
                              'dki_biggest.nii.gz')
    f = nibabel.load(segment_file)
    segment_map = f.get_data()
    return segment_map

In [4]:
def get_mk_map(subject, dataLoc):
    '''
    Returns nibabel matrix of
    the whole brain mean kurtosis image
    '''
    mk_file = os.path.join(dataLoc,
                         subject,
                         'DKI',
                         'kmean.nii')
    f = nibabel.load(mk_file)
    mk_map = f.get_data()
    return mk_map

In [5]:
def get_mean_MK_for_nuclei(segment_map, mk_map, cortex):
    '''
    Returns mean of the MK, and the volume 
    of the thalamic segment
    in connection to sepcific cortex
    '''
    nuclei_dict = {"LPFC":1,
                  "LTC":2,
                  "MPFC":3,
                  "MTC":4,
                  "OCC":5,
                  "OFC":6,
                  "PC":7,
                  "SMC":8}
    nucleiNum = nuclei_dict[cortex]
    cortexMap = segment_map == nucleiNum # returns boolean matrix of the matching cortex
    segment_volume = numpy.sum(cortexMap) # count number of True (voxels of the matching cortex)
    
    segment_mk = cortexMap * mk_map # returns mk_map within the thalamic nucleus of matching cortex
    mk_mean = segment_mk[segment_mk!=0].mean() # non-zero mean

    return mk_mean, segment_volume

In [6]:
def get_short_side(side):
    if side == 'left':
        side_s = 'lh'
    else:
        side_s = 'rh'
    return side_s

In [7]:
def get_cortex_volume(subject, side, dataLoc, cortex):
    '''
    Returns the volume of the given cortex ROI
    '''
    side_s = get_short_side(side)
    cortexROI = os.path.join(dataLoc,
                            subject,
                            'ROI',
                            '{side_s}_{cortex}.nii.gz'.format(side_s=side_s,
                                                             cortex=cortex))
    cortexVolume = get_volume(cortexROI)
    return cortexVolume

In [8]:
def get_connectivity_for_nuclei(subject, side, dataLoc, cortex):
    '''
    Returns the sum of connectivity
    in the thalamic segment
    '''

    
    side_s = get_short_side(side)
    connectivity_file = os.path.join(dataLoc, 
                                    subject,
                                    'segmentation',
                                    side,
                                    'seeds_to_{side_s}_{cortex}.nii.gz'.format(side_s=side_s,
                                                                                cortex=cortex))
    totalConnectivity = get_volume(connectivity_file)
    return totalConnectivity
    

In [9]:
def get_volume(imgFile):
    return nibabel.load(imgFile).get_data().sum()

In [10]:
def thalamic_mk(subject,side,dataLoc):
    '''
    Returns a pandas dataframe,
    containing MK of thalamic segments 
    for a given subject
    '''
    group = subject[:3]
    segment_map = get_segment_map(subject, side, dataLoc)
    mk_map = get_mk_map(subject, dataLoc)
    side_s = get_short_side(side)
    
    df = pandas.DataFrame()
    for cortex in ["LPFC", "LTC", "MPFC", "MTC", "OCC", "OFC", "PC", "SMC"]:
        totalConnectivity = get_connectivity_for_nuclei(subject, side, dataLoc, cortex)
        cortexVolume = get_cortex_volume(subject, side, dataLoc, cortex)
        mk_mean, segment_volume = get_mean_MK_for_nuclei(segment_map, mk_map, cortex)
        
        cortex_df = pandas.DataFrame(data = [[mk_mean, 
                                          segment_volume,
                                          totalConnectivity,
                                          cortexVolume,
                                          cortex, 
                                          side, 
                                          group]], 
                                 index=[subject], 
                                 columns=['MK',
                                          'nucleiVolume',
                                          'totalConnectivity',
                                          'cortexVolume',
                                          'cortex',
                                          'side',
                                          'group'])
        df = pandas.concat([df, cortex_df])
    
    df['relativeConnectivity'] = df.totalConnectivity / df.totalConnectivity.sum()
    df['thalamucVolume'] = get_volume(os.path.join(dataLoc, subject, 'ROI', side_s+'_thalamus.nii.gz'))
    return df

In [11]:
def main():
    '''
    Estimates the mean kurtosis in the thalamic segments
    for all subjects.
    The thalamic segments is obtained from the connectivity-based segmentation
    from FSL.
    
    It returns mk_data.csv containing all subjects' data.
    
    '''
    allData = pandas.DataFrame()
    for subject in subjects:
        for side in ['left','right']:
            subjectDf = thalamic_mk(subject, side)
            allData = pandas.concat([allData, subjectDf])
    
    allData.to_csv('mk_data.csv')

In [20]:
def main_parallel(subjects, ncpus=20):
    '''
    Estimates the mean kurtosis in the thalamic segments
    for all subjects.
    The thalamic segments is obtained from the connectivity-based segmentation
    from FSL.
    
    It returns mk_data.csv containing all subjects' data.
    
    '''

    
    ppservers=()
    job_server = pp.Server(ncpus, ppservers=ppservers, secret='ccncserver')
    print "Starting pp with", job_server.get_ncpus(), "workers"
    
    
    start_time = time.time()
    
    jobs=[]
    for side in ['left','right']:
        job = [(subject, 
                 job_server.submit(thalamic_mk,
                                   (subject,side,dataLoc),
                                   (get_segment_map,
                                    get_mk_map,
                                    get_short_side,
                                    get_volume,
                                    get_connectivity_for_nuclei,
                                    get_cortex_volume,
                                    get_mean_MK_for_nuclei),
                                   ("os","pandas","nibabel","numpy"))) for subject in subjects]
        jobs = jobs+job
    
    
    allData = pandas.DataFrame()
    for command, job in jobs:
        print command, "is completed"
        subjectDf = job()
        allData = pandas.concat([allData, subjectDf])
    
    print "Time elapsed: ", time.time() - start_time, "s"
    job_server.print_stats()
    
    print allData
    allData.to_csv('mk_data.csv')

In [19]:
main_parallel(subjects,20)

Starting pp with 20 workers
NOR54_SSR is completed
NOR56_YIW is completed
NOR54_SSR is completed
NOR56_YIW is completed
/ccnc_bin/venv/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
Time elapsed:  22.0210921764 s
Job execution statistics:
 job count | % of all jobs | job time sum | time per job | job server
         4 |        100.00 |      87.8601 |    21.965022 | local
Time elapsed since server creation 22.0216529369
0 active tasks, 20 cores

                       MK  nucleiVolume  totalConnectivity  cortexVolume  \
NOR54_SSR   1.09940481186           101            2121739         26594   
NOR54_SSR  0.809105396271            31            1544127         39806   
NOR54_SSR  0.963177204132           125            2356440         29168   
NOR54_SSR  0.928298413754           111            1663457         13981   
NOR54_SSR  0.683056890965            11             257482         215

```
if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        formatter_class=argparse.RawDescriptionHelpFormatter,
        description=textwrap.dedent('''\
            {codeName} : Estimates mean kurtosis in the thalamic nuclei
            ===========================================================
            '''.format(codeName=os.path.basename(__file__))))

    parser.add_argument(
        '-s', '--subjects',
        help='subject list',
        nargs='+')

    parser.add_argument(
        '-side', '--side',
        help='side')
    
    args = parser.parse_args()    
    thalamic_mk(args)
```